In [1]:
%matplotlib inline
import importlib

In [4]:
import utils2; importlib.reload(utils2)
from utils2 import *

In [5]:
from bcolz_array_iterator import BcolzArrayIterator
from tqdm import tqdm

In [6]:
limit_mem()

In [23]:
path = '/home/ubuntu/nbs/dataset/train/'
dpath = '/home/ubuntu/nbs/dataset'

In [4]:
from gensim.models import word2vec

In [5]:
w2v_path = '/home/ubuntu/nbs/dataset/GoogleNews-vectors-negative300'

In [6]:
# Converting word vector's from bin format to txt format
model = word2vec.KeyedVectors.load_word2vec_format(w2v_path+'.bin', binary = True)
model.save_word2vec_format(w2v_path+'.txt', binary=False)

In [21]:
import numpy as np
import pickle
def parse_w2v(l):
    i=l.index(' ')
    return l[:i], np.fromstring(l[i+1:-2], 'float32', sep=' ')

In [28]:
# with io.open(w2v_path+'.txt', "r", encoding="utf-8") as my_file:
lines = open(w2v_path+'.txt', "r", encoding="utf-8").readlines()

In [29]:
lines

['3000000 300\n',
 '</s> 0.001129 -0.000896 0.000319 0.001534 0.001106 -0.001404 -0.000031 -0.000420 -0.000576 0.001076 -0.001022 -0.000618 -0.000755 0.001404 -0.001640 -0.000633 0.001633 -0.001007 -0.001266 0.000652 -0.000416 -0.001076 0.001526 -0.000275 0.000140 0.001572 0.001358 -0.000832 -0.001404 0.001579 0.000254 -0.000732 -0.000105 -0.001167 0.001579 0.000656 -0.000660 0.000003 0.001129 0.000427 -0.000370 -0.001152 0.001266 -0.000004 0.000265 -0.000402 0.000141 -0.000034 0.000759 -0.000519 -0.000071 0.000603 -0.000507 -0.001625 -0.000437 -0.000992 -0.001221 -0.000322 0.000069 -0.001167 -0.000511 0.001411 0.000336 -0.000475 -0.001373 0.000366 -0.001442 -0.000607 0.000801 0.001129 -0.000835 -0.001160 0.000916 0.000523 -0.000328 0.001595 -0.001579 -0.000357 0.000496 0.001015 -0.001099 -0.000166 -0.000142 -0.000261 0.001259 0.000039 0.000169 -0.001030 0.001610 0.000629 0.000418 -0.001350 0.000349 0.001144 -0.001205 -0.001183 0.000950 0.000061 0.000011 -0.000668 0.001244 0.000690 0.0

In [30]:
w2v_list = list(map(parse_w2v, lines[1:]))

In [40]:
pickle.dump(w2v_list, open(path+'../w2vl.pkl', 'wb'))

In [25]:
w2v_list = pickle.load(open(path+'../w2vl.pkl', 'rb'))

In [34]:
w2v_dict = dict(w2v_list)

In [35]:
words,vectors = zip(*w2v_list)

In [43]:
np.corrcoef(w2v_dict['bill'], w2v_dict['BILL'])

array([[ 1.        ,  0.27939364],
       [ 0.27939364,  1.        ]])

In [44]:
# Create Reverse mapping from lower cased word to embedding's
lc_w2v = {w.lower(): w2v_dict[w] for w in reversed(words)}

In [47]:
# LOL no embedding exist's for lc_w2v["aalekh"]

In [49]:
fpath = '/home/ubuntu/nbs/dataset/imagenet_class_index.json' # Remember to add all these files, I am providing path to